In [ ]:
import math
import random
import pygame
from pygame.locals import *

# Define cell sizes
CELL_SIZE = 30
GRID_SIZE = 15
WINDOW_SIZE = (CELL_SIZE * GRID_SIZE, CELL_SIZE * GRID_SIZE)

class CellType:
    EMPTY = 0
    BUILDING = 1
    HOUSE = 2
    DELIVERY_POINT = 3
    VEHICLE = 4

class CityGrid:
    def __init__(self, size):
        self.size = size
        self.grid = [[CellType.EMPTY for _ in range(size)] for _ in range(size)]
        self.vehicles = []

    def set_cell(self, x, y, cell_type):
        if 0 <= x < self.size and 0 <= y < self.size:
            self.grid[y][x] = cell_type
        else:
            print("Coordinates out of range.")

    def get_cell(self, x, y):
        if 0 <= x < self.size and 0 <= y < self.size:
            return self.grid[y][x]
        else:
            return None

    def generate_random_cost(self):
        return random.randint(1, 20)

    def euclidean_distance(self, x1, y1, x2, y2):
        return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

    def a_star(self, start, goal):
        open_set = {start}
        came_from = {}
        g_score = {start: 0}
        f_score = {start: self.euclidean_distance(start[0], start[1], goal[0], goal[1])}

        while open_set:
            current = min(open_set, key=lambda x: f_score.get(x, math.inf))
            if current == goal:
                return self.reconstruct_path(came_from, goal)
            open_set.remove(current)

            neighbors = [(current[0] + dx, current[1] + dy) for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]]
            for neighbor in neighbors:
                if neighbor[0] < 0 or neighbor[0] >= self.size or neighbor[1] < 0 or neighbor[1] >= self.size:
                    continue
                if self.get_cell(*neighbor) == CellType.BUILDING:
                    continue
                tentative_g_score = g_score[current] + self.generate_random_cost()
                if tentative_g_score < g_score.get(neighbor, math.inf):
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + self.euclidean_distance(neighbor[0], neighbor[1], goal[0], goal[1])
                    open_set.add(neighbor)

        return None

    def reconstruct_path(self, came_from, current):
        path = [current]
        while current in came_from:
            current = came_from[current]
            path.append(current)
        path.reverse()
        return path

    def update_vehicle_positions(self, positions):
        self.vehicles = positions

# Function to execute path
def execute_path(city, start, goal):
    # Run A* algorithm
    path = city.a_star(start, goal)
    if path:
        print("Path found from start to goal:", start, "->", goal)
        print("Path:", path)
        # Execute path here (not implemented in this example)
        return path
    else:
        print("No path found from start to goal:", start, "->", goal)
        return []

# Function to draw grid
def draw_grid(screen, city):
    screen.fill((255, 255, 255))
    for y in range(GRID_SIZE):
        for x in range(GRID_SIZE):
            rect = pygame.Rect(x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE)
            if city.get_cell(x, y) == CellType.BUILDING:
                pygame.draw.rect(screen, (0, 0, 0), rect)
            elif city.get_cell(x, y) == CellType.HOUSE:
                pygame.draw.rect(screen, (0, 0, 255), rect)
            elif city.get_cell(x, y) == CellType.DELIVERY_POINT:
                pygame.draw.rect(screen, (0, 255, 0), rect)
            elif (x, y) in city.vehicles:
                pygame.draw.rect(screen, (255, 0, 0), rect, 3)
            pygame.draw.rect(screen, (0, 0, 0), rect, 1)
    pygame.display.flip()

# Function to animate the robot's movement
def animate_path(screen, city, path):
    for i in range(len(path) - 1):
        draw_grid(screen, city)
        pygame.draw.line(screen, (255, 255, 0), (path[i][0] * CELL_SIZE + CELL_SIZE // 2, path[i][1] * CELL_SIZE + CELL_SIZE // 2),
                         (path[i + 1][0] * CELL_SIZE + CELL_SIZE // 2, path[i + 1][1] * CELL_SIZE + CELL_SIZE // 2), 3)
        pygame.display.flip()
        pygame.time.delay(500)  # Delay between each step
    draw_grid(screen, city)
    pygame.display.flip()

# Main function
def main():
    pygame.init()
    screen = pygame.display.set_mode(WINDOW_SIZE)
    pygame.display.set_caption("Delivery Robot Simulation")

    city = CityGrid(GRID_SIZE)

    # Define buildings
    city.set_cell(3, 3, CellType.BUILDING)
    city.set_cell(4, 3, CellType.BUILDING)
    city.set_cell(5, 3, CellType.BUILDING)
    city.set_cell(6, 3, CellType.BUILDING)
    city.set_cell(7, 3, CellType.BUILDING)
    city.set_cell(8, 3, CellType.BUILDING)

    # Define houses
    city.set_cell(10, 5, CellType.HOUSE)
    city.set_cell(11, 5, CellType.HOUSE)
    city.set_cell(12, 5, CellType.HOUSE)

    # Define initial vehicle positions
    initial_vehicle_positions = [(8, 7), (9, 8), (10, 9)]
    city.update_vehicle_positions(initial_vehicle_positions)

    # Generate 5 random delivery locations
    delivery_locations = []
    for _ in range(5):
        delivery_x = random.randint(0, city.size - 1)
        delivery_y = random.randint(0, city.size - 1)
        while city.get_cell(delivery_x, delivery_y) != CellType.EMPTY:
            delivery_x = random.randint(0, city.size - 1)
            delivery_y = random.randint(0, city.size - 1)
        delivery_locations.append((delivery_x, delivery_y))

    # Start robot's delivery process
    for i, delivery_location in enumerate(delivery_locations):
        print("Robot waiting for delivery...")
        start = initial_vehicle_positions[0] if i == 0 else delivery_locations[i - 1]
        goal = delivery_location
        print("Assigning delivery item to robot for delivery to:", goal)
        path = execute_path(city, start, goal)
        animate_path(screen, city, path)

    print("All deliveries completed!")

    # Main loop
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == QUIT:
                running = False

    pygame.quit()

if __name__ == "__main__":
    main()


Robot waiting for delivery...
Assigning delivery item to robot for delivery to: (12, 1)
Path found from start to goal: (8, 7) -> (12, 1)
Path: [(8, 7), (8, 6), (8, 5), (9, 5), (9, 4), (9, 3), (10, 3), (11, 3), (12, 3), (12, 2), (12, 1)]
Robot waiting for delivery...
Assigning delivery item to robot for delivery to: (10, 1)
Path found from start to goal: (12, 1) -> (10, 1)
Path: [(12, 1), (12, 2), (11, 2), (10, 2), (10, 1)]
Robot waiting for delivery...
Assigning delivery item to robot for delivery to: (10, 12)
Path found from start to goal: (10, 1) -> (10, 12)
Path: [(10, 1), (10, 2), (11, 2), (11, 3), (11, 4), (11, 5), (11, 6), (11, 7), (11, 8), (11, 9), (11, 10), (11, 11), (11, 12), (10, 12)]
Robot waiting for delivery...
Assigning delivery item to robot for delivery to: (9, 8)
Path found from start to goal: (10, 12) -> (9, 8)
Path: [(10, 12), (9, 12), (9, 11), (9, 10), (9, 9), (9, 8)]
Robot waiting for delivery...
Assigning delivery item to robot for delivery to: (6, 8)
Path found f

In [ ]:
import math
import random
import pygame
from pygame.locals import *

# Define cell sizes
CELL_SIZE = 30
GRID_SIZE = 15
WINDOW_SIZE = (CELL_SIZE * GRID_SIZE, CELL_SIZE * GRID_SIZE)

class CellType:
    EMPTY = 0
    BUILDING = 1
    HOUSE = 2
    DELIVERY_POINT = 3
    VEHICLE = 4

class CityGrid:
    def __init__(self, size):
        self.size = size
        self.grid = [[CellType.EMPTY for _ in range(size)] for _ in range(size)]
        self.vehicles = []

    def set_cell(self, x, y, cell_type):
        if 0 <= x < self.size and 0 <= y < self.size:
            self.grid[y][x] = cell_type
        else:
            print("Coordinates out of range.")

    def get_cell(self, x, y):
        if 0 <= x < self.size and 0 <= y < self.size:
            return self.grid[y][x]
        else:
            return None

    def generate_random_cost(self):
        return random.randint(1, 20)

    def euclidean_distance(self, x1, y1, x2, y2):
        return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

    def a_star(self, start, goal):
        open_set = {start}
        came_from = {}
        g_score = {start: 0}
        f_score = {start: self.euclidean_distance(start[0], start[1], goal[0], goal[1])}

        while open_set:
            current = min(open_set, key=lambda x: f_score.get(x, math.inf))
            if current == goal:
                return self.reconstruct_path(came_from, goal)
            open_set.remove(current)

            neighbors = [(current[0] + dx, current[1] + dy) for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]]
            for neighbor in neighbors:
                if neighbor[0] < 0 or neighbor[0] >= self.size or neighbor[1] < 0 or neighbor[1] >= self.size:
                    continue
                if self.get_cell(*neighbor) == CellType.BUILDING:
                    continue
                tentative_g_score = g_score[current] + self.generate_random_cost()
                if tentative_g_score < g_score.get(neighbor, math.inf):
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + self.euclidean_distance(neighbor[0], neighbor[1], goal[0], goal[1])
                    open_set.add(neighbor)

        return None

    def reconstruct_path(self, came_from, current):
        path = [current]
        while current in came_from:
            current = came_from[current]
            path.append(current)
        path.reverse()
        return path

    def update_vehicle_positions(self, positions):
        self.vehicles = positions

# Function to execute path
def execute_path(city, start, goal):
    # Run A* algorithm
    path = city.a_star(start, goal)
    if path:
        print("Path found from start to goal:", start, "->", goal)
        print("Path:", path)
        # Execute path here (not implemented in this example)
        return path
    else:
        print("No path found from start to goal:", start, "->", goal)
        return []

# Function to draw grid
def draw_grid(screen, city):
    screen.fill((255, 255, 255))
    for y in range(GRID_SIZE):
        for x in range(GRID_SIZE):
            rect = pygame.Rect(x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE)
            if city.get_cell(x, y) == CellType.BUILDING:
                pygame.draw.rect(screen, (0, 0, 0), rect)
            elif city.get_cell(x, y) == CellType.HOUSE:
                pygame.draw.rect(screen, (0, 0, 255), rect)
            elif city.get_cell(x, y) == CellType.DELIVERY_POINT:
                pygame.draw.rect(screen, (0, 255, 0), rect)
            elif (x, y) in city.vehicles:
                pygame.draw.rect(screen, (255, 0, 0), rect)
    pygame.display.flip()

# Function to animate the robot's movement
def animate_path(screen, city, path):
    for i in range(len(path) - 1):
        draw_grid(screen, city)
        pygame.draw.line(screen, (255, 255, 0), (path[i][0] * CELL_SIZE + CELL_SIZE // 2, path[i][1] * CELL_SIZE + CELL_SIZE // 2),
                         (path[i + 1][0] * CELL_SIZE + CELL_SIZE // 2, path[i + 1][1] * CELL_SIZE + CELL_SIZE // 2), 2)
        pygame.display.flip()
        pygame.time.delay(500)  # Delay between each step
    draw_grid(screen, city)
    pygame.display.flip()

# Main function
def main():
    pygame.init()
    screen = pygame.display.set_mode(WINDOW_SIZE)
    pygame.display.set_caption("Delivery Robot Simulation")

    city = CityGrid(GRID_SIZE)

    # Define buildings
    city.set_cell(3, 3, CellType.BUILDING)
    city.set_cell(4, 3, CellType.BUILDING)
    city.set_cell(5, 3, CellType.BUILDING)
    city.set_cell(6, 3, CellType.BUILDING)
    city.set_cell(7, 3, CellType.BUILDING)
    city.set_cell(8, 3, CellType.BUILDING)

    # Define houses
    city.set_cell(10, 5, CellType.HOUSE)
    city.set_cell(11, 5, CellType.HOUSE)
    city.set_cell(12, 5, CellType.HOUSE)

    # Define initial vehicle positions
    initial_vehicle_positions = [(8, 7), (9, 8), (10, 9)]
    city.update_vehicle_positions(initial_vehicle_positions)

    # Generate 5 random delivery locations
    delivery_locations = []
    for _ in range(5):
        delivery_x = random.randint(0, city.size - 1)
        delivery_y = random.randint(0, city.size - 1)
        while city.get_cell(delivery_x, delivery_y) != CellType.EMPTY:
            delivery_x = random.randint(0, city.size - 1)
            delivery_y = random.randint(0, city.size - 1)
        delivery_locations.append((delivery_x, delivery_y))

    # Start robot's delivery process
    for i, delivery_location in enumerate(delivery_locations):
        print("Robot waiting for delivery...")
        start = initial_vehicle_positions[0] if i == 0 else delivery_locations[i - 1]
        goal = delivery_location
        print("Assigning delivery item to robot for delivery to:", goal)
        path = execute_path(city, start, goal)
        animate_path(screen, city, path)

    print("All deliveries completed!")

    # Main loop
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == QUIT:
                running = False

    pygame.quit()

if __name__ == "__main__":
    main()
